In [ ]:
import numpy as np
import tensorflow as tf
import os
import cv2
import time
import collections


import tqdm

In [ ]:
from tftools import data
import sys
import os
episode_model_save_location = '/data/robotics/pick_episodes/models'
episode_data_save_location = '/data/robotics/pick_episodes/data'
pyrobo_path = '/home/karthik/code/pyrobo/'
sys.path.append(os.path.join(pyrobo_path, 'network_models'))
sys.path.insert(0, '/home/karthik/code/pytools/src/')

from tftools import network as nw
import multicam_net
import trainer

In [ ]:
def load_all_training_data(train_data_location):
    train_files = [os.path.join(train_data_location, f) for f in os.listdir(train_data_location)
                   if f.endswith('.tfrecords')]
    img_cnt = 0
    for train_fl in train_files:
        img_cnt += data.TFDataReader(train_fl).count()
    train_inp = np.zeros([2, img_cnt, 224, 224, 3], dtype=np.uint8)
    train_labels = np.zeros([img_cnt, ], dtype=np.int32)
    train_states = np.zeros([img_cnt, 6], dtype=np.float32)
    train_values = np.zeros([img_cnt, ], dtype=np.float32)
    end_states = np.zeros([img_cnt, ], dtype=np.int32)
    idx = 0
    for train_fl in train_files:
        reader = data.TFDataReader(train_fl)
        fds = reader.readall((["left_image", data.IMG_TYP],
                              ["right_image", data.IMG_TYP],
                              ['motor', data.INT_TYP],
                              ['direction', data.INT_TYP],
                              ['motor_state', data.FLT_TYP]))
        value_idx = 0.
        value_count = len(fds)
        for fd in fds:
            train_inp[0, idx, ...] = multicam_net.resize_to_model(fd['left_image'])
            train_inp[1, idx, ...] = multicam_net.resize_to_model(fd['right_image'])
            motor = int(fd['motor'][0])
            direction = int(fd['direction'][0])
            train_states[idx, :] = np.array(fd['motor_state'])
            label = trainer.motor_2_label(motor, direction)
            train_values[idx] = value_idx/value_count
            train_labels[idx] = label
            idx += 1
            value_idx+=1
        end_states[idx-1] = 1

    return ((train_inp, train_labels, train_states, train_values, end_states), img_cnt)



In [ ]:
((train_inp, train_labels, train_states, train_values, end_states), img_cnt) = load_all_training_data(episode_data_save_location)

In [ ]:
class MultiImageAutEnModel(nw.Network):
    def __init__(self):
        self.left_img = tf.placeholder(tf.float32, shape=[None, 224, 224, 3])
        self.right_img = tf.placeholder(tf.float32, shape=[None, 224, 224, 3])
        self.img = tf.concat(values=[self.left_img, 
                                     self.right_img], axis=3)
        self.layers = dict({'img':self.img})
        self.trainable = True
        self.initializer =  tf.contrib.layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32)
        self.setup()        
        
    def setup(self):
        (self.feed('img')
             .conv(7, 7, 64*2, 2, 2,  name='conv1_7x7_s2', trainable=False, group=2)
             .max_pool(3, 3, 2, 2,  name='pool1_3x3_s2')
             .lrn(2, 2e-05, 0.75,   name='pool1_norm1')
             .conv(1, 1, 64*2, 1, 1,  name='conv2_3x3_reduce', trainable=False, group=2)
             .conv(3, 3, 192*2, 1, 1, name='conv2_3x3', trainable=False, group=2)
             .lrn(2, 2e-05, 0.75,   name='conv2_norm2')
             .max_pool(3, 3, 2, 2,  name='pool2_3x3_s2'))
        
        (self.feed('pool2_3x3_s2')
             .conv(1, 1, 64*2, 1, 1, name='inception_3a_3x3_reduce', group=2)
             .conv(3, 3, 32*2, 1, 1, name='inception_3a_3x3', group=2) 
             .conv(3, 3, 32, 1, 1, name='inception_3b_3x3')) 
        
        (self.feed('inception_3b_3x3')
            .fc(4096, name="fc1")
            .fc(512, name="fc3")
            .fc(128, name="bn"))

        (self.feed('bn')
            .fc(128, name='bc1')
            .fc(384, name='bc2')
            .fc(384*2, name='out'))
        
        (self.feed("conv2_3x3")
        .spatial_softmax(name='ss'))
        
        (self.feed('bn')
            .fc(128, name='val_1')
            .fc(16, name='val_2')
            .fc(1, name='val_out'))

        

In [ ]:
net = MultiImageAutEnModel()


In [ ]:
lr = tf.placeholder(dtype=tf.float32)

In [ ]:
loss = tf.reduce_mean(tf.square(net.layers['ss'] - net.layers['out']))
optimizer = tf.train.GradientDescentOptimizer(lr)
train_op = optimizer.minimize(loss)

val_optimizer = tf.train.RMSPropOptimizer(lr)

values = tf.placeholder(shape=(None,), dtype=tf.float32)
value_loss = tf.reduce_mean(tf.square(values  - net.layers['val_out']))
value_train = val_optimizer.minimize(value_loss)

In [ ]:
sess = tf.Session()

In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
net.load('/data/image/models/googlenet.npy', sess, True)

In [ ]:
last_100_1 = collections.deque(100*[100], 100)
last_100_2 = collections.deque(100*[100], 100)

In [ ]:
LR = 0.1

In [ ]:
tr = tqdm.tnrange(1000)
for step in tr:
    idxes = np.random.randint(0, img_cnt, size=16)
    left_im = train_inp[0, idxes, ...]
    right_im = train_inp[1, idxes, ...]
    batch_data = {
                    net.left_img: left_im,
                    net.right_img: right_im,
                values: train_values[idxes],
                lr: LR}
    [_, lv1] = sess.run([value_train, value_loss], batch_data)
    batch_data = {
                net.left_img: left_im,
                net.right_img: right_im,
                lr: LR}
    [_, lv2] = sess.run([train_op, loss], batch_data)

    last_100_1.append(lv1)
#     last_100_2.append(lv2)
    tr.set_postfix(loss1=np.mean(last_100_1), loss2=np.mean(last_100_2))
b


In [ ]:
net.initializer

In [ ]:
tf.contrib.layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32)

In [ ]:
cnt = 0
tot = 0
bel0pt1 = 0
for l in open('/home/karthik/code/images/simul_learning_test/final_loss_single').readlines():
    v = eval(l.strip())
    cnt+=1
    tot+=v[0]
    if v[0] < 0.1:
        bel0pt1+=1

In [ ]:
net.layers['conv1_7x7_s2']

In [ ]:
import vpnotebook

In [ ]:
from vpython import *


In [ ]:
box()